In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from tensorflow.keras.optimizers import RMSprop
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
import numpy as np
from tensorflow.keras import layers
import tensorflow as tf

In [ ]:
train_generator = ImageDataGenerator(rescale = 1/255.0) 

test_generator = ImageDataGenerator(rescale = 1/255.0 , validation_split = 0.5) 

In [ ]:
# train_Data = train_generator.flow_from_directory('/content/drive/MyDrive/capstone project /computer vision_face_emotion_recognisation/dataset/train' ,target_size = (48,48), batch_size= 128 , class_mode = 'binary')

In [ ]:
train_Data = train_generator.flow_from_directory('/content/drive/MyDrive/capstone project /computer vision_face_emotion_recognisation/dataset/train' ,target_size = (224,224), batch_size= 128 , class_mode = 'binary')

Found 10420 images belonging to 7 classes.


In [ ]:
validation_Data = test_generator.flow_from_directory('/content/drive/MyDrive/capstone project /computer vision_face_emotion_recognisation/dataset/test' ,target_size = (224,224), batch_size= 128 , class_mode = 'binary' , subset = 'validation')

Found 3587 images belonging to 7 classes.


In [ ]:
test_Data = test_generator.flow_from_directory('/content/drive/MyDrive/capstone project /computer vision_face_emotion_recognisation/dataset/test' ,target_size = (224,224), batch_size= 128 , class_mode = 'binary',subset = 'training')

Found 3591 images belonging to 7 classes.


In [ ]:
base_model = VGG16(input_shape = (48,48,3) , include_top = False)

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
data_augmentation = keras.Sequential(
    [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),]
)

In [ ]:
inputs = keras.Input(shape=(48, 48, 3))
x = data_augmentation(inputs) 

x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(7)(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 sequential (Sequential)     (None, 48, 48, 3)         0         
                                                                 
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 7)                 3591      
                                                             

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

epochs = 20
model.fit(train_Data, epochs=epochs, validation_data=validation_Data)

Epoch 1/20
82/82 [==============================] - 299s 4s/step - loss: 1.2192 - accuracy: 0.4335 - val_loss: 4.0838 - val_accuracy: 0.1985
Epoch 2/20
82/82 [==============================] - 298s 4s/step - loss: 1.1121 - accuracy: 0.4782 - val_loss: 4.3200 - val_accuracy: 0.2208
Epoch 3/20
82/82 [==============================] - 298s 4s/step - loss: 1.0918 - accuracy: 0.4927 - val_loss: 4.3960 - val_accuracy: 0.2411
Epoch 4/20
82/82 [==============================] - 298s 4s/step - loss: 1.0757 - accuracy: 0.5021 - val_loss: 4.7500 - val_accuracy: 0.2403
Epoch 5/20
82/82 [==============================] - 298s 4s/step - loss: 1.0604 - accuracy: 0.5135 - val_loss: 4.9405 - val_accuracy: 0.2662
Epoch 6/20
82/82 [==============================] - 298s 4s/step - loss: 1.0450 - accuracy: 0.5230 - val_loss: 4.9569 - val_accuracy: 0.2635
Epoch 7/20
82/82 [==============================] - 298s 4s/step - loss: 1.0363 - accuracy: 0.5327 - val_loss: 5.3561 - val_accuracy: 0.2540
Epoch 8/20
82

In [ ]:
base_model.trainable = True
model.summary()

model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate=1e-5),  # Low learning rate
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

epochs = 10
model.fit(train_Data, epochs=epochs, validation_data=validation_Data)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 sequential (Sequential)     (None, 48, 48, 3)         0         
                                                                 
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 7)                 3591      
                                                             

In [ ]:
#mobilenetv2

In [ ]:
model1 = tf.keras.applications.MobileNetV2()

14548992/14536120 [==============================] - 0s 0us/step


In [ ]:
model1.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [ ]:
base_input = model1.layers[0].input

In [ ]:
base_output = model1.layers[-2].output

In [ ]:
final_output = keras.layers.Dense(128,activation = 'relu')(base_output)
final_output = keras.layers.Dropout(0.5)(final_output)

In [ ]:
final_output = keras.layers.Dense(64,activation = 'relu')(final_output)
final_output = keras.layers.Dense(7,activation = 'softmax')(final_output)

In [ ]:
final_model = keras.Model(inputs = base_input , outputs = final_output)

In [ ]:
final_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [ ]:
final_model.compile(loss = 'sparse_categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])

In [ ]:
final_model.fit(train_Data, epochs=25, validation_data=validation_Data)

Epoch 1/25
82/82 [==============================] - 3136s 38s/step - loss: 1.2182 - accuracy: 0.4848 - val_loss: 21.0154 - val_accuracy: 0.1335
Epoch 2/25
82/82 [==============================] - 124s 2s/step - loss: 0.8808 - accuracy: 0.6480 - val_loss: 23.1295 - val_accuracy: 0.1447
Epoch 3/25
82/82 [==============================] - 123s 1s/step - loss: 0.7331 - accuracy: 0.7082 - val_loss: 29.0860 - val_accuracy: 0.2713
Epoch 4/25
82/82 [==============================] - 123s 2s/step - loss: 0.6355 - accuracy: 0.7498 - val_loss: 20.3287 - val_accuracy: 0.1347
Epoch 5/25
82/82 [==============================] - 123s 2s/step - loss: 0.5770 - accuracy: 0.7823 - val_loss: 18.0095 - val_accuracy: 0.1567
Epoch 6/25
82/82 [==============================] - 124s 2s/step - loss: 0.4837 - accuracy: 0.8180 - val_loss: 18.9830 - val_accuracy: 0.2919
Epoch 7/25
82/82 [==============================] - 123s 2s/step - loss: 0.4480 - accuracy: 0.8335 - val_loss: 32.4406 - val_accuracy: 0.2473
Epoc

In [ ]:
final_model.save('mobile_net_model.h5')